掌握回归模型的评估及超参数调优
参数与超参数

    参数：使用最小二乘法或者梯度下降法等最优化算法优化出来的数
    超参数：无法使用最小二乘法或者梯度下降法等最优化算法优化出来的数
    对比：
    序号 	       参数 	                 超参数
    1 	  定义了可使用的模型 	     用于帮助估计模型参数
    2 	  从数据估计中得到 	          模型外部的配置，其值无法从数据中估计
    3 	  不由编程者手动设置 	     由人工指定，可以使用启发式设置
    4 	  保存为学习模型的一部分 	    被调整为给定的预测建模问题

采用何种方式对损失函数进行迭代优化，这是机器学习的一大主题之一，当一个机器学习问题有了具体的模型和评估策略，所有的机器学习问题都可以形式化为一个最优化问题。这也是为什么我们说优化理论和凸优化算法等学科是机器学习一大支柱的原因所在。从纯数学的角度来看，所有的数学模型尽管形式不一，各有头面，但到最后几乎到可以归约为最优化问题。所以，有志于奋战在机器学习和深度学习领域的各位，学好最优化，责无旁贷啊。
      要说机器学习和深度学习的优化算法，梯度下降必然是核心所在。神经网络发展至今，优化算法层出不穷，但大底是出不了梯度下降的框框架架。这一篇笔记，笔者就和大家一起学习和回顾深度学习中常用的优化算法。在前面手动搭建神经网络的代码实践中，我们对于损失函数的优化采用了一般的梯度下降法，所以本篇总结就从梯度下降法开始。
梯度下降法 Gradient Descent
![jupyter](./gd.png)
      想必大家对于梯度下降是很熟悉了，选择负梯度方向进行参数更新算是常规操作了。话不多说，对于多层神经网络如何执行梯度下降：

In [9]:
def update_parameters_with_gd(parameters, grads, learning_rate):
    """
    Update parameters using one step of gradient descent

    Arguments:
    parameters -- python dictionary containing your parameters to be updated:
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    grads -- python dictionary containing your gradients to update each parameters:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    learning_rate -- the learning rate, scalar.
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    L = len(parameters) // 2 # number of layers in the neural networks
    # Update rule for each parameter
    for l in range(L):  
        parameters['W' + str(l+1)] = parameters['W' + str(l+1)] - learning_rate * grads['dW' + str(l+1)]
        parameters['b' + str(l+1)] = parameters['b' + str(l+1)] - learning_rate * grads['db' + str(l+1)]  
    return parameters

在上述代码中，我们传入含有权值和偏置的字典、梯度字段和更新的学习率作为参数，按照开头的公式编写权值更新代码，一个简单的多层网络的梯度下降算法就写出来了。
小批量梯度下降法 mini-batch Gradient Descent

      在工业数据环境下，直接对大数据执行梯度下降法训练往往处理速度缓慢，这时候将训练集分割成小一点的子集进行训练就非常重要了。这个被分割成的小的子集就叫做 mini-batch，意为小批量。对每一个小批量同时执行梯度下降会大大提高训练效率。在实际利用代码实现的时候，小批量梯度下降算法通常包括两个步骤：充分打乱数据（shuffle）和分组组合数据(partition)。如下图所示。
![jupyter](./minibatch.png)

#shuffle
![jupyter](./shuffle.png)
#partition

In [10]:
def random_mini_batches(X, Y, mini_batch_size = 64, seed = 0):
    """
    Creates a list of random minibatches from (X, Y)

    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (1 for blue dot / 0 for red dot), of shape (1, number of examples)
    mini_batch_size -- size of the mini-batches, integer

    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """

    np.random.seed(seed)        
    m = X.shape[1]                 
    mini_batches = []    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((1,m))    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) 
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, 0:mini_batch_size]
        mini_batch_Y = shuffled_Y[:, 0:mini_batch_size]

        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, 0: m-mini_batch_size*math.floor(m/mini_batch_size)]
        mini_batch_Y = shuffled_Y[:, 0: m-mini_batch_size*math.floor(m/mini_batch_size)]

        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)    
    return mini_batches

小批量梯度下降的实现思路非常清晰，先打乱数据在分组数据，需要注意的细节在于最后一个小批量所含的训练样本数，通常而言最后一个小批量会少于前面批量所含样本数。
随机梯度下降 Stochastic Gradient Descent
      当小批量所含的训练样本数为 1 的时候，小批量梯度下降法就变成了随机梯度下降法（SGD）。SGD虽然以单个样本为训练单元训练速度会很快，但牺牲了向量化运算所带来的便利性，在较大数据集上效率并不高。
      我们可以看一下梯度下降和随机梯度下降在实现上的差异：

In [ ]:
# GD
X = data_input
Y = labels
parameters = initialize_parameters(layers_dims)
for i in range(0, num_iterations):    # Forward propagation
    a, caches = forward_propagation(X, parameters)    # Compute cost.
    cost = compute_cost(a, Y)    # Backward propagation.
    grads = backward_propagation(a, caches, parameters)    # Update parameters.
    parameters = update_parameters(parameters, grads)

# SGDX = data_input
Y = labels
parameters = initialize_parameters(layers_dims)
for i in range(0, num_iterations):    
    for j in range(0, m):        # Forward propagation
        a, caches = forward_propagation(X[:,j], parameters)        # Compute cost
        cost = compute_cost(a, Y[:,j])        # Backward propagation
        grads = backward_propagation(a, caches, parameters)        # Update parameters.
        parameters = update_parameters(parameters, grads)

所以，从本质上看，梯度下降法、小批量梯度下降法和随机梯度下降法，并没有区别。唯一的区别就在于它们执行一次训练过程所需要用到的训练样本数。梯度下降法用到的是全集训练数据，随机梯度下降则是单个样本数据，而小批量则是介于二者之间。
带动量的梯度下降法（momentum）

![jupyter](./sgdm.png)
正如上图中看到的一样，我们假设梯度下降的横向为参数 W 的下降方向，而偏置 b 的下降方向为纵轴，我们总是希望在纵轴上的震荡幅度小一点，学习速度慢一点，而在横轴上学习速度快一点，无论是小批量梯度下降还是随机梯度下降，好像都不能避免这个问题。为了解决这个问题，带动量的梯度下降法来了。带动量的梯度下降考虑历史梯度的加权平均值作为速率进行优化。执行公式如下：
![jupyter](./sgdm1.png)
根据上述公式编写带动量的梯度下降法实现代码：

In [12]:
def update_parameters_with_momentum(parameters, grads, v, beta, learning_rate):
    """
    Update parameters using Momentum

    Arguments:
    parameters -- python dictionary containing your parameters:
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    grads -- python dictionary containing your gradients for each parameters:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    v -- python dictionary containing the current velocity:
                    v['dW' + str(l)] = ...
                    v['db' + str(l)] = ...
    beta -- the momentum hyperparameter, scalar
    learning_rate -- the learning rate, scalar

    Returns:
    parameters -- python dictionary containing your updated parameters 
    v -- python dictionary containing your updated velocities
    """

    L = len(parameters) // 2 # number of layers in the neural networks

    # Momentum update for each parameter
    for l in range(L):        # compute velocities
        v['dW' + str(l+1)] = beta * v['dW' + str(l+1)] + (1-beta)* grads['dW' + str(l+1)]
        v['db' + str(l+1)] = beta * v['db' + str(l+1)] + (1-beta)* grads['db' + str(l+1)]        # update parameters
        parameters['W' + str(l+1)] = parameters['W' + str(l+1)] - learning_rate*v['dW' + str(l+1)]
        parameters['b' + str(l+1)] = parameters['b' + str(l+1)] - learning_rate*v['db' + str(l+1)] 
    return parameters, v

实现带动量的梯度下降的关键点有两个：一是动量是考虑历史梯度进行梯度下降的，二是这里的需要指定的超参数变成了两个：一个是学习率 learning_rate，一个是梯度加权参数beta。
Adam算法
      Adam 全称为 Adaptive Moment Estimation，是在带动量的梯度下降法的基础上融合了一种称为 RMSprop（加速梯度下降）的算法而成的。相较于带动量的梯度下降法，无论是RMSprop 还是 Adam，其中的改进思路都在于如何让横轴上的学习更快以及让纵轴上的学习更慢。RMSprop 和 Adam 在带动量的梯度下降法的基础上，引入了平方梯度，并对速率进行了偏差纠正。具体计算公式如下：
![jupyter](./adam.png)

In [13]:
# 实现代码如下：

def update_parameters_with_adam(parameters, grads, v, s, t, learning_rate = 0.01,
                                beta1 = 0.9, beta2 = 0.999,  epsilon = 1e-8):
    """
    Update parameters using Adam

    Arguments:
    parameters -- python dictionary containing your parameters:
                    parameters['W' + str(l)] = Wl
                    parameters['b' + str(l)] = bl
    grads -- python dictionary containing your gradients for each parameters:
                    grads['dW' + str(l)] = dWl
                    grads['db' + str(l)] = dbl
    v -- Adam variable, moving average of the first gradient, python dictionary
    s -- Adam variable, moving average of the squared gradient, python dictionary
    learning_rate -- the learning rate, scalar.
    beta1 -- Exponential decay hyperparameter for the first moment estimates 
    beta2 -- Exponential decay hyperparameter for the second moment estimates 
    epsilon -- hyperparameter preventing division by zero in Adam updates

    Returns:
    parameters -- python dictionary containing your updated parameters 
    v -- Adam variable, moving average of the first gradient, python dictionary
    s -- Adam variable, moving average of the squared gradient, python dictionary
    """

    L = len(parameters) // 2                 
    v_corrected = {}                        
    s_corrected = {}                         

    # Perform Adam update on all parameters
    for l in range(L):
        v["dW" + str(l+1)] = beta1 * v["dW" + str(l+1)] + (1 - beta1) * grads['dW'+str(l+1)]
        v["db" + str(l+1)] = beta1 * v["db" + str(l+1)] + (1 - beta1) * grads['db'+str(l+1)]        # Compute bias-corrected first moment estimate. Inputs: "v, beta1, t". Output: "v_corrected".   
        v_corrected["dW" + str(l+1)] = v["dW" + str(l+1)] / (1 - beta1**t)
        v_corrected["db" + str(l+1)] = v["db" + str(l+1)] / (1 - beta1**t)        # Moving average of the squared gradients. Inputs: "s, grads, beta2". Output: "s".
        s["dW" + str(l+1)] = beta2 * s["dW" + str(l+1)] + (1 - beta2) * (grads["dW" + str(l+1)])**2
        s["db" + str(l+1)] = beta2 * s["db" + str(l+1)] + (1 - beta2) * (grads["db" + str(l+1)])**2


        # Compute bias-corrected second raw moment estimate. Inputs: "s, beta2, t". Output: "s_corrected".
        s_corrected["dW" + str(l+1)] = s["dW" + str(l+1)] / (1 - beta2**t)
        s_corrected["db" + str(l+1)] = s["db" + str(l+1)] / (1 - beta2**t)        # Update parameters. Inputs: "parameters, learning_rate, v_corrected, s_corrected, epsilon". Output: "parameters".

        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * v_corrected["dW" + str(l+1)] / (np.sqrt(s_corrected["dW" + str(l+1)]) + epsilon)
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * v_corrected["db" + str(l+1)] / (np.sqrt(s_corrected["db" + str(l+1)]) + epsilon)    
    return parameters, v, s

 除了以上这些算法，还有一些像 Adadelta 之类的算法我们没有提到，有需要了解的同学可以自行查找相关资料。最后用一个图来展示各种优化算法的效果：
![jupyter](./adam.gif)

(5) 对模型超参数进行调优(调参)：                             
在刚刚的讨论中，我们似乎对模型的优化都是对模型算法本身的改进，比如：岭回归对线性回归的优化在于在线性回归的损失函数中加入L2正则化项从而牺牲无偏性降低方差。但是，大家是否想过这样的问题：在L2正则化中参数$\lambda$应该选择多少？是0.01、0.1、还是1？到目前为止，我们只能凭经验或者瞎猜，能不能找到一种方法找到最优的参数$\lambda$？事实上，找到最佳参数的问题本质上属于最优化的内容，因为从一个参数集合中找到最佳的值本身就是最优化的任务之一，我们脑海中浮现出来的算法无非就是：梯度下降法、牛顿法等无约束优化算法或者约束优化算法，但是在具体验证这个想法是否可行之前，我们必须先认识两个最本质概念的区别。                                     
   - 参数与超参数：                            
   我们很自然的问题就是岭回归中的参数$\lambda$和参数w之间有什么不一样？事实上，参数w是我们通过设定某一个具体的$\lambda$后使用类似于最小二乘法、梯度下降法等方式优化出来的，我们总是设定了$\lambda$是多少后才优化出来的参数w。因此，类似于参数w一样，使用最小二乘法或者梯度下降法等最优化算法优化出来的数我们称为参数，类似于$\lambda$一样，我们无法使用最小二乘法或者梯度下降法等最优化算法优化出来的数我们称为超参数。                                       
   模型参数是模型内部的配置变量，其值可以根据数据进行估计。                       
      - 进行预测时需要参数。                     
      - 它参数定义了可使用的模型。                        
      - 参数是从数据估计或获悉的。                       
      - 参数通常不由编程者手动设置。                     
      - 参数通常被保存为学习模型的一部分。                      
      - 参数是机器学习算法的关键，它们通常由过去的训练数据中总结得出 。                          
   模型超参数是模型外部的配置，其值无法从数据中估计。
      - 超参数通常用于帮助估计模型参数。
      - 超参数通常由人工指定。
      - 超参数通常可以使用启发式设置。
      - 超参数经常被调整为给定的预测建模问题。                            
   我们前面(4)部分的优化都是基于模型本身的具体形式的优化，那本次(5)调整的内容是超参数，也就是取不同的超参数的值对于模型的性能有不同的影响。                             
   - 网格搜索GridSearchCV()：                
   网格搜索：https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html?highlight=gridsearchcv#sklearn.model_selection.GridSearchCV                         
   网格搜索结合管道：https://scikit-learn.org/stable/auto_examples/compose/plot_compare_reduction.html?highlight=gridsearchcv                              
   网格搜索的思想非常简单，比如你有2个超参数需要去选择，那你就把所有的超参数选择列出来分别做排列组合。举个例子：$\lambda = 0.01,0.1,1.0$和$\alpha = 0.01,0.1,1.0$,你可以做一个排列组合，即：{[0.01,0.01],[0.01,0.1],[0.01,1],[0.1,0.01],[0.1,0.1],[0.1,1.0],[1,0.01],[1,0.1],[1,1]}  ，然后针对每组超参数分别建立一个模型，然后选择测试误差最小的那组超参数。换句话说，我们需要从超参数空间中寻找最优的超参数，很像一个网格中找到一个最优的节点，因此叫网格搜索。                         
   - 随机搜索 RandomizedSearchCV() ：               
   https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html?highlight=randomizedsearchcv#sklearn.model_selection.RandomizedSearchCV                           
   网格搜索相当于暴力地从参数空间中每个都尝试一遍，然后选择最优的那组参数，这样的方法显然是不够高效的，因为随着参数类别个数的增加，需要尝试的次数呈指数级增长。有没有一种更加高效的调优方式呢？那就是使用随机搜索的方式，这种方式不仅仅高校，而且实验证明，随机搜索法结果比稀疏化网格法稍好(有时候也会极差，需要权衡)。参数的随机搜索中的每个参数都是从可能的参数值的分布中采样的。与网格搜索相比，这有两个主要优点：        
      - 可以独立于参数数量和可能的值来选择计算成本。                 
      - 添加不影响性能的参数不会降低效率。            
下面我们使用SVR的例子，结合管道来进行调优：

In [1]:
# 我们先来对未调参的SVR进行评价： 
from sklearn.svm import SVR     # 引入SVR类
from sklearn.pipeline import make_pipeline   # 引入管道简化学习流程
from sklearn.preprocessing import StandardScaler # 由于SVR基于距离计算，引入对数据进行标准化的类
from sklearn.model_selection import GridSearchCV  # 引入网格搜索调优
from sklearn.model_selection import cross_val_score # 引入K折交叉验证
from sklearn import datasets
import numpy as np


boston = datasets.load_boston()     # 返回一个类似于字典的类
X = boston.data
y = boston.target
features = boston.feature_names
pipe_SVR = make_pipeline(StandardScaler(), SVR())
score1 = cross_val_score(estimator=pipe_SVR, X = X, y = y, scoring = 'r2', cv = 10)       # 10折交叉验证
print("CV accuracy: %.3f +/- %.3f" % ((np.mean(score1)), np.std(score1)))

CV accuracy: 0.187 +/- 0.649


GridSearchCV的sklearn官方网址：

http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV

GridSearchCV，它存在的意义就是自动调参，只要把参数输进去，就能给出最优化的结果和参数。但是这个方法适合于小数据集，一旦数据的量级上去了，很难得出结果。这个时候就是需要动脑筋了。数据量比较大的时候可以使用一个快速调优的方法——坐标下降。它其实是一种贪心算法：拿当前对模型影响最大的参数调优，直到最优化；再拿下一个影响最大的参数调优，如此下去，直到所有的参数调整完毕。这个方法的缺点就是可能会调到局部最优而不是全局最优，但是省时间省力，巨大的优势面前，还是试一试吧，后续可以再拿bagging再优化。

通常算法不够好，需要调试参数时必不可少。比如SVM的惩罚因子C，核函数kernel，gamma参数等，对于不同的数据使用不同的参数，结果效果可能差1-5个点，sklearn为我们提供专门调试参数的函数grid_search。

In [2]:
# 下面我们使用网格搜索来对SVR调参：
from sklearn.pipeline import Pipeline
pipe_svr = Pipeline([("StandardScaler",StandardScaler()),("svr",SVR())])
param_range = [0.0001,0.001,0.01,0.1,1.0,10.0,100.0,1000.0]
 # 注意__是指两个下划线，一个下划线会报错的

param_grid = [{"svr__C":param_range,"svr__kernel":["linear"]}, {"svr__C":param_range,"svr__gamma":param_range,"svr__kernel":["rbf"]}]

# param_grid={'kernel':['linear','rbf'],'C':[0.001,0.01,0.1,1,10,100,1000],
#             'gamma':[0.001,0.01,0.1,1,10,100,1000]}
gs = GridSearchCV(estimator=pipe_svr, param_grid = param_grid, scoring = 'r2', cv = 10)       # 10折交叉验证
gs = gs.fit(X,y)
print("网格搜索最优得分：", gs.best_score_)
print("网格搜索最优参数组合：\n", gs.best_params_)

网格搜索最优得分： 0.6081303070817237
网格搜索最优参数组合：
 {'svr__C': 1000.0, 'svr__gamma': 0.001, 'svr__kernel': 'rbf'}


In [3]:
# 下面我们使用随机搜索来对SVR调参：
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform  # 引入均匀分布设置参数
pipe_svr = Pipeline([("StandardScaler",StandardScaler()), ("svr",SVR())])
distributions = dict(svr__C=uniform(loc=1.0, scale=4),    # 构建连续参数的分布
                     svr__kernel=["linear","rbf"],                                   # 离散参数的集合
                    svr__gamma=uniform(loc=0, scale=4))

rs = RandomizedSearchCV(estimator=pipe_svr, param_distributions = distributions, scoring = 'r2', cv = 10)# 10折交叉验证
rs = rs.fit(X,y)
print("随机搜索最优得分：",rs.best_score_)
print("随机搜索最优参数组合：\n",rs.best_params_)

随机搜索最优得分： 0.302402169705699
随机搜索最优参数组合：
 {'svr__C': 1.2433913013577484, 'svr__gamma': 1.1173508136621315, 'svr__kernel': 'linear'}


经过我们不懈的努力，从收集数据集并选择合适的特征、选择度量模型性能的指标、选择具体的模型并进行训练以优化模型到评估模型的性能并调参，我们认识到了如何使用sklearn构建简单回归模型。在本章的最后，我们会给出一个具体的案例，整合回归的内容。下面我们来看看机器学习另外一类大问题：分类。与回归一样，分类问题在机器学习的地位非常重要，甚至有的地方用的比回归问题还要多，因此分类问题是十分重要的！
